In [1]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import matplotlib.pyplot as plt
import cv2
import numpy as np

%matplotlib inline

1.6.0 True
2.18.1
10.1
GCC 7.3


## Download original weights

In [3]:
! mkdir -p /mmdetection/checkpoints/
! wget -nc https://download.openmmlab.com/mmdetection/v2.0/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-b5f6da5e.pth \
       -O /mmdetection/checkpoints/vfnet.pth

File ‘/mmdetection/checkpoints/vfnet.pth’ already there; not retrieving.


In [2]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Choose to use a config and initialize the detector
config = '/mmdetection/configs/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py'
# Setup a checkpoint file to load
checkpoint = '/mmdetection/checkpoints/vfnet.pth'

In [ ]:
# initialize the detector
model = init_detector(config, checkpoint, device='cuda:1')

## Change the config

In [4]:
from mmcv import Config
cfg = Config.fromfile(config)

from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'CocoDataset'
cfg.classes = ('fawn', 'reindeer')
cfg.data_root = '/mmdetection/data/'
#cfg.train_pipeline.insert(
#    4,
#    dict(
#        type='CutOut',
#        n_holes=(0, 4),
#        cutout_ratio=[(0.01, 0.008), (0.005, 0.01), (0.005, 0.01), (0.01, 0.01)],
#    )
#)
#cfg.train_pipeline.insert(
#    4,
#    dict(
#        type='RandomAffine',
#        n_holes=(0, 4),
#        cutout_ratio=[(0.01, 0.008), (0.005, 0.01), (0.005, 0.01), (0.01, 0.01)],
#    )
#)

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 4

cfg.data.test.type = 'CocoDataset'
cfg.data.test.data_root = cfg.data_root
cfg.data.test.ann_file = 'test/coco_annotations.json'
cfg.data.test.img_prefix = 'test/data/'
cfg.data.test.classes = cfg.classes

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = cfg.data_root
cfg.data.train.ann_file = 'train/coco_annotations.json'
cfg.data.train.img_prefix = 'train/data/'
cfg.data.train.classes = cfg.classes

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = cfg.data_root
cfg.data.val.ann_file = 'all_data/coco_annotations.json'
cfg.data.val.img_prefix = 'all_data/images/'
cfg.data.val.classes = cfg.classes


cfg.model.bbox_head.num_classes = len(cfg.classes)

# We can still use the pre-trained model
cfg.load_from = checkpoint

# Set up working dir to save files and logs.
cfg.work_dir = './deers_exps/'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.002 / 8
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = ['bbox', ]
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [1, ]


# We can initialize the logger for training and have a look
# at the final config used for training
#print(f'Config:\n{cfg.pretty_text}')

## Let's train

In [ ]:
import mmcv
import os.path as osp
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'),
)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
print(datasets[0].CLASSES)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
('fawn', 'reindeer')
loading annotations into memory...


2021-11-19 13:07:06,486 - mmdet - INFO - load checkpoint from local path: /mmdetection/checkpoints/vfnet.pth


Done (t=0.24s)
creating index...
index created!


2021-11-19 13:07:08,256 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.vfnet_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([2, 256, 3, 3]).
size mismatch for bbox_head.vfnet_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([2]).
2021-11-19 13:07:08,264 - mmdet - INFO - Start running, host: root@0751bf428dc3, work_dir: /mmdetection/notebooks/deers_exps
2021-11-19 13:07:08,265 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
